In [1]:
!pip --q install datasets
!pip --q install accelerate
!pip --q install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Audio
import numpy as np
import torch
import pandas as pd
from datasets import DatasetDict,Dataset
import soundfile as sf
import os
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!git clone https://github.com/Mahmood-Anaam/VitsModelSplit.git

Cloning into 'VitsModelSplit'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 166 (delta 99), reused 120 (delta 56), pack-reused 0
Receiving objects: 100% (166/166), 20.97 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (99/99), done.




---



In [5]:
from VitsModelSplit.vits_model import VitsModel
from VitsModelSplit.PosteriorDecoderModel import PosteriorDecoderModel
from VitsModelSplit.feature_extraction import VitsFeatureExtractor

from transformers import AutoTokenizer, HfArgumentParser, set_seed
from VitsModelSplit.Arguments import DataTrainingArguments, ModelArguments, VITSTrainingArguments


In [6]:
model = VitsModel.from_pretrained("facebook/mms-tts-ara",cache_dir="./")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-ara",cache_dir="./")
feature_extractor = VitsFeatureExtractor()

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-tts-ara were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [7]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, VITSTrainingArguments))
json_file = os.path.abspath('/content/VitsModelSplit/finetune_config_ara.json')
model_args, data_args, training_args = parser.parse_json_file(json_file = json_file)



---



In [8]:
posterior_decoder_model = PosteriorDecoderModel(config = model.config,
                                                posterior_encoder = model.posterior_encoder,
                                                decoder = model.decoder
                                                )
posterior_decoder_model.train()
posterior_decoder_model

PosteriorDecoderModel(
  (posterior_encoder): VitsPosteriorEncoder(
    (conv_pre): Conv1d(513, 192, kernel_size=(1,), stride=(1,))
    (wavenet): VitsWaveNet(
      (in_layers): ModuleList(
        (0-15): 16 x ParametrizedConv1d(
          192, 384, kernel_size=(5,), stride=(1,), padding=(2,)
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): _WeightNorm()
            )
          )
        )
      )
      (res_skip_layers): ModuleList(
        (0-14): 15 x ParametrizedConv1d(
          192, 384, kernel_size=(1,), stride=(1,)
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): _WeightNorm()
            )
          )
        )
        (15): ParametrizedConv1d(
          192, 192, kernel_size=(1,), stride=(1,)
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): _WeightNorm()
            )
          )
        )
      )
      (dropout

In [9]:
from  VitsModelSplit.dataset_features_collector import FeaturesCollectionDataset

dataset_dir = '/content/drive/MyDrive/FeaturesCollectionDataset'
full_generation_sample_index = 0
full_generation_dataset = FeaturesCollectionDataset(dataset_dir = os.path.join(dataset_dir,'full_generation'))
full_generation_sample = full_generation_dataset[full_generation_sample_index]

set_seed(42)
with torch.no_grad():
  full_generation =model(
      input_ids =full_generation_sample["input_ids"],
      attention_mask=full_generation_sample["attention_mask"],
      speaker_id=full_generation_sample["speaker_id"]
      )
  full_generation_waveform = full_generation.waveform.numpy().reshape(-1)


Audio(full_generation_waveform, rate=model.config.sampling_rate)



---



In [10]:
dataset_dir = '/content/drive/MyDrive/FeaturesCollectionDataset'

posterior_decoder_model.trainer(
                              train_dataset_dir = os.path.join(dataset_dir,'train'),
                              eval_dataset_dir = os.path.join(dataset_dir,'eval'),
                              full_generation_dir = os.path.join(dataset_dir,'full_generation'),
                              feature_extractor = VitsFeatureExtractor(),
                              training_args = training_args,
                              full_generation_sample_index= 0,
                              project_name = "Posterior_Decoder_Finetuning",
                              wandbKey = "782b6a6e82bbb5a5348de0d3c7d40d1e76351e79",
                              )

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: modelasg. Use `wandb login --relogin` to force relogin


TRAINIG - batch 0, waveform torch.Size([10, 83040, 1]), step_loss_mel 0.6283440589904785, lr 1.99975e-05... 
TRAINIG - batch 1, waveform torch.Size([10, 135680, 1]), step_loss_mel 0.6494752764701843, lr 1.99975e-05... 
TRAINIG - batch 2, waveform torch.Size([10, 52036, 1]), step_loss_mel 0.6461055874824524, lr 1.99975e-05... 
TRAINIG - batch 3, waveform torch.Size([10, 62720, 1]), step_loss_mel 0.5772804021835327, lr 1.99975e-05... 
TRAINIG - batch 4, waveform torch.Size([6, 46560, 1]), step_loss_mel 0.6477117538452148, lr 1.99975e-05... 
VALIDATION - batch 0, waveform torch.Size([10, 67200, 1]), step_loss_mel 0.6477117538452148 ... 
VALIDATION - batch 1, waveform torch.Size([4, 67520, 1]), step_loss_mel 0.5747188329696655 ... 
TRAINIG - batch 5, waveform torch.Size([10, 138560, 1]), step_loss_mel 0.5918751358985901, lr 1.99975e-05... 
TRAINIG - batch 0, waveform torch.Size([10, 83040, 1]), step_loss_mel 0.5546226501464844, lr 1.99950003125e-05... 
TRAINIG - batch 1, waveform torch.Siz



---



In [11]:
model_new = VitsModel.from_pretrained("facebook/mms-tts-ara",cache_dir="./")
model_new.posterior_encoder = posterior_decoder_model.posterior_encoder
model_new.decoder = posterior_decoder_model.decoder

Some weights of the model checkpoint at facebook/mms-tts-ara were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [12]:
from  VitsModelSplit.dataset_features_collector import FeaturesCollectionDataset

dataset_dir = '/content/drive/MyDrive/FeaturesCollectionDataset'
full_generation_sample_index = 0
full_generation_dataset = FeaturesCollectionDataset(dataset_dir = os.path.join(dataset_dir,'full_generation'))
full_generation_sample = full_generation_dataset[full_generation_sample_index]

set_seed(42)
with torch.no_grad():
  full_generation =model_new(
      input_ids =full_generation_sample["input_ids"],
      attention_mask=full_generation_sample["attention_mask"],
      speaker_id=full_generation_sample["speaker_id"]
      )
  full_generation_waveform = full_generation.waveform.numpy().reshape(-1)


Audio(full_generation_waveform, rate=model_new.config.sampling_rate)